## Gregory M. Eirich
## Example 
## Lab #6

# Lab 6 Examples


## 1. Run a naive ("pooled") OLS regression on the panel data.  Tell we how you expect your Xs to affect your Y and why.   Interpret your results.

## 2. Run a first differences regression on the same model in Question 1.  Interpret your results.  Do you draw a different conclusion than in Question 1?  Explain.


In [4]:
pip install linearmodels

Note: you may need to restart the kernel to use updated packages.


In [2]:
from __future__ import division
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import matplotlib.pyplot as plt
import linearmodels as lm

from linearmodels import PanelOLS
from linearmodels import RandomEffects
from linearmodels import FirstDifferenceOLS

## Use the GSS Panel in Folder for Lab 6 on Courseworks.

In [3]:
os.chdir('C:/Users/gme2101/Desktop/Data Analysis Data/') # change working directory

g = pd.read_csv("panel-for-R.csv") ## choose "panel-for-R.csv" and more information on variables are here: http://sda.berkeley.edu/sdaweb/analysis/?dataset=gss06panelw3 ##

g.head()

,idnum,panelwave,ballot,form,formwt,oversamp,sample,panstat_2,panstat_3,mar1,...,wtpan12,wtpan123,wtpannr12,wtpannr123,xmarsex,xmovie,xnorcsiz,year,yearval,zodiac
0,9,1,3,2,1,1,9,1,1,5.0,...,0.414689,0.487828,0.435503,0.470575,1.0,2.0,1.0,2006.0,NaN,9.0
1,9,2,3,2,1,1,9,1,1,5.0,...,0.414689,0.487828,0.435503,0.470575,1.0,1.0,1.0,2008.0,NaN,9.0
2,9,3,3,2,1,1,9,1,1,1.0,...,0.414689,0.487828,0.435503,0.470575,1.0,1.0,1.0,2010.0,NaN,9.0
3,10,1,1,1,1,1,9,1,1,5.0,...,0.829377,0.858741,0.766632,0.828371,1.0,NaN,1.0,2006.0,NaN,3.0
4,10,2,1,1,1,1,9,1,1,5.0,...,0.829377,0.858741,0.766632,0.828371,1.0,NaN,1.0,2008.0,NaN,3.0


## Making some recodes on race, interviewer's race and opinions on affirmative action.

In [4]:
## this is if the person is black or not

conditions = [
    (g['race'] ==2) ,
     ((g['race'] ==1) | (g['race']==3))]
choices = [1,0]
g['black'] = np.select(conditions, choices, default=np.nan)

In [5]:
g.black.value_counts()

0.0    4139
1.0     673
Name: black, dtype: int64

In [6]:
## this is if the interviewer is black or not

conditions = [
    (g['intrace1'] ==2) ,
     ((g['intrace1'] ==1) | (g['intrace1']==3))]
choices = [1,0]
g['intblack'] = np.select(conditions, choices, default=np.nan)

In [7]:
g.intblack.value_counts()

0.0    3952
1.0     629
Name: intblack, dtype: int64

## Reverse code how people feel about your opinion "Do you favor preference in hiring and promotion strongly or not strongly for blacks? If opposes: Do you oppose preference in hiring and promotion strongly or not strongly?" ranging from strong support to strong opposition

In [8]:
## this is how people feel about What about your opinion "Do you favor preference in hiring and promotion strongly or not strongly for blacks? If opposes: Do you oppose preference in hiring and promotion strongly or not strongly?" ranging from strong support to strong opposition

g['r_affact'] = 5-g['affrmact'] 

## now, higher scores mean someone is more supportive of affirmative action

In [10]:
g.r_affact.value_counts()

1.0    1620
2.0     860
4.0     292
3.0     242
Name: r_affact, dtype: int64

## 1. Run a naive ("pooled") OLS regression on the panel data.  Tell we how you expect your Xs to affect your Y and why.   Interpret your results.

## The question is net of race, does being interviewed in person by an African American make Americans more inclined to support affirmative action hiring?

In [11]:
lm_ols = smf.ols(formula = 'r_affact ~ black + intblack', data = g).fit()
print (lm_ols.summary())

                            OLS Regression Results                            
Dep. Variable:               r_affact   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     130.6
Date:                Tue, 25 Jun 2019   Prob (F-statistic):           5.18e-55
Time:                        12:21:07   Log-Likelihood:                -3847.4
No. Observations:                2862   AIC:                             7701.
Df Residuals:                    2859   BIC:                             7719.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.5992      0.019     83.013      0.0

## As can be seen, African Americans are more inclined to be in favor of affirmative action, but also, regardless of race, if someone is interviewed by an African American, they become more supportive too.

## 2. Run a first differences regression on the same model in Question 1.  Interpret your results.  Do you draw a different conclusion than in Question 1?  Explain.

In [12]:
g = g.set_index(['idnum', 'panelwave'])
lm_fd = FirstDifferenceOLS.from_formula('r_affact ~ black + intblack', g)
res_fd = lm_fd.fit(cov_type='clustered', cluster_entity=True)
print(res_fd)

C:\Users\gme2101\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                     FirstDifferenceOLS Estimation Summary                      
Dep. Variable:               r_affact   R-squared:                        0.0014
Estimator:         FirstDifferenceOLS   R-squared (Between):              0.0288
No. Observations:                1561   R-squared (Within):               0.0037
Date:                Tue, Jun 25 2019   R-squared (Overall):              0.0267
Time:                        12:21:33   Log-likelihood                   -2141.0
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1.1293
Entities:                        1272   P-value                           0.3235
Avg Obs:                       2.2500   Distribution:                  F(2,1559)
Min Obs:                       1.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             0.5898
                            

## When we look to see what happens if someone switches from having an non-African-American interviewer to having an African-American interviewer, they do become slightly more supportive of affirmative action, but not statistically significantly so (p=0.29).  By the way, if someone switches from identifying as non-African-American to then identifying as African-American (which likely doesn't happen much), they become a bit more supportive of affirmative action, but not statistically significantly so (p=0.82).